In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine

In [2]:
df = pd.read_csv('radio_songs.csv')

In [3]:
df.head()

,user,abba,ac/dc,adam green,aerosmith,afi,air,alanis morissette,alexisonfire,alicia keys,...,timbaland,tom waits,tool,tori amos,travis,trivium,u2,underoath,volbeat,yann tiersen
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,33,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,42,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,51,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,62,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 1. Collaborative Filtering

### A. Recommend 10 songs to users who have listened to 'u2' and 'pink floyd'.
#### Use item-item collaborative filtering to find songs that are similar using spatial distance with cosine.

#### Checking Data

In [4]:
print('We have 100 users with 284 songs.')

We have 100 users with 284 songs.


In [5]:
df['u2'].value_counts()

0    96
1     4
Name: u2, dtype: int64

In [6]:
df['pink floyd'].value_counts()

0    97
1     3
Name: pink floyd, dtype: int64

In [7]:
df1 = df.copy()
df1['Target'] = df['u2']+df['pink floyd']

In [8]:
df1['Target'].value_counts()

0    93
1     7
Name: Target, dtype: int64

In [9]:
print('After checking, we can see that there is no user who have listened to u2 and pink floyd. So we will find the users that have listened us or pink floyd to build the recommnedation system.')

After checking, we can see that there is no user who have listened to u2 and pink floyd. So we will find the users that have listened us or pink floyd to build the recommnedation system.


#### Calculate Similarity

In [10]:
# df1.columns.get_loc("u2")
# df1.columns.get_loc("pink floyd")

In [11]:
X = df.copy()
X.drop(['user','u2','pink floyd'],axis = 1, inplace = True)
X.head()

,abba,ac/dc,adam green,aerosmith,afi,air,alanis morissette,alexisonfire,alicia keys,all that remains,...,three days grace,timbaland,tom waits,tool,tori amos,travis,trivium,underoath,volbeat,yann tiersen
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
u2_vector = df['u2']
pf_vector = df['pink floyd']

In [13]:
u2_sim = []

for i in range(0,282):
    item1 = u2_vector
    item2 = X.iloc[:,i]
    similarity = 1 - cosine(item1, item2)
    u2_sim.append(similarity)

/Users/apple/opt/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [14]:
u2_sim_df = pd.DataFrame({'song': X.columns,
                         'similarity': u2_sim})
u2_sim_df.sort_values(by='similarity', ascending = False).iloc[:10]

,song,similarity
219,robbie williams,0.500000
176,misfits,0.500000
111,green day,0.433013
71,depeche mode,0.408248
201,peter fox,0.377964
90,enter shikari,0.353553
164,madonna,0.353553
136,johnny cash,0.353553
77,dire straits,0.353553
148,kelly clarkson,0.353553


In [15]:
pf_sim = []

for i in range(0,282):
    item1 = pf_vector
    item2 = X.iloc[:,i]
    similarity = 1 - cosine(item1, item2)
    pf_sim.append(similarity)

/Users/apple/opt/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [16]:
pf_sim_df = pd.DataFrame({'song': X.columns,
                         'similarity': pf_sim})
pf_sim_df.sort_values(by='similarity', ascending = False).iloc[:10]

,song,similarity
105,genesis,0.577350
65,david bowie,0.408248
156,led zeppelin,0.408248
104,funeral for a friend,0.408248
116,hans zimmer,0.408248
206,queen,0.408248
232,sonic syndicate,0.408248
59,coldplay,0.348155
168,maria mena,0.333333
42,bob dylan,0.333333


In [17]:
ten_songs = [list(u2_sim_df.sort_values(by='similarity', ascending = False).iloc[:5,0])+
     list(pf_sim_df.sort_values(by='similarity', ascending = False).iloc[:5,0])]
ten_songs

[['robbie williams',
  'misfits',
  'green day',
  'depeche mode',
  'peter fox',
  'genesis',
  'david bowie',
  'led zeppelin',
  'funeral for a friend',
  'hans zimmer']]

### B. Find user most similar to user 1606. Use user-user collaborative filtering with cosine similarity. List the recommended songs for user 1606 
#### (Hint: find the songs listened to by the most similar user).

In [18]:
df[df['user'] == 1606]

,user,abba,ac/dc,adam green,aerosmith,afi,air,alanis morissette,alexisonfire,alicia keys,...,timbaland,tom waits,tool,tori amos,travis,trivium,u2,underoath,volbeat,yann tiersen
99,1606,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
user_X = df.copy()
user_X.drop(labels=99,axis = 0, inplace = True)
user_X.shape

(99, 285)

In [20]:
user_y = df.iloc[99,1:]

In [21]:
user_sim = []

for i in range(0,99):
    item1 = user_y
    item2 = user_X.iloc[i,1:]
    similarity = 1 - cosine(item1, item2)
    user_sim.append(similarity)

/Users/apple/opt/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [22]:
user_sim_df = pd.DataFrame({'user': user_X['user'],
                         'similarity': user_sim})
user_sim_df.sort_values(by='similarity', ascending = False).iloc[:5]

,user,similarity
65,1144,0.277350
8,144,0.223607
79,1334,0.150756
89,1509,0.144338
45,890,0.129099


In [23]:
user1144 = df[df.user == 1144].T
user1144[user1144.iloc[:,0] == 1]

,65
beastie boys,1
bob dylan,1
bob marley & the wailers,1
david bowie,1
elvis presley,1
eric clapton,1
johnny cash,1
pearl jam,1
pink floyd,1
the beatles,1


In [24]:
print('The recommended songs for user 1606 are: ') 
list(user1144[user1144.iloc[:,0] == 1].index)

The recommended songs for user 1606 are: 


['beastie boys',
 'bob dylan',
 'bob marley & the wailers',
 'david bowie',
 'elvis presley',
 'eric clapton',
 'johnny cash',
 'pearl jam',
 'pink floyd',
 'the beatles',
 'the doors',
 'the rolling stones',
 'tom waits']

### C. How many of the recommended songs has already been listened to by user 1606?

In [25]:
user1606 = df[df.user == 1606].T
user1606[user1606.iloc[:,0] == 1]

,99
abba,1
elvis presley,1
frank sinatra,1
the beatles,1


In [26]:
user1144_song = list(user1144[user1144.iloc[:,0] == 1].index)
user1606_song = list(user1606[user1606.iloc[:,0] == 1].index)

In [27]:
listened = []

for i in range(0, len(user1606_song)):
    check = user1606_song[i] in user1144_song
    if check == True:
        listened.append(user1606_song[i])

In [28]:
listened

['elvis presley', 'the beatles']

In [29]:
print('User 1606 has already listened ',listened)

User 1606 has already listened  ['elvis presley', 'the beatles']


### D. Use a combination of user-item approach to build a recommendation score for each song for each user using the following steps for each user-

#### 1. For each song for the user row, get the top 10 similar songs and their similarity score.

In [30]:
song_sim_list = []
song_sim_cal = []

for song in range(1,285):
    song_name = df.columns[song]
    song_sim_sub = []
    
    song_X = df.copy()
    song_X.drop(['user',song_name],axis = 1, inplace = True)
    song_y = df[song_name]
    
    for other_song in range(0,283):  
        item1 = song_y
        item2 = song_X.iloc[:,other_song]
        similarity = 1 - cosine(item1, item2)
        song_sim_sub.append(similarity)
    
    song_sim_sub_df = pd.DataFrame({'song': song_X.columns,'similarity': song_sim_sub})
    song_sim_sub_df = song_sim_sub_df.sort_values(by='similarity', ascending = False)

    song_sim_list.append(list(song_sim_sub_df.iloc[:10,0]))
    song_sim_cal.append(list(song_sim_sub_df.iloc[:10,1]))

/Users/apple/opt/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [31]:
## Checking the result:
song_sim_list[df1.columns.get_loc("u2")-1]

['robbie williams',
 'misfits',
 'green day',
 'depeche mode',
 'peter fox',
 'dire straits',
 'kelly clarkson',
 'madonna',
 'johnny cash',
 'enter shikari']

In [32]:
## Checking the result:
song_sim_cal[df1.columns.get_loc("u2")-1]

[0.5,
 0.5,
 0.4330127018922193,
 0.40824829046386313,
 0.3779644730092272,
 0.35355339059327373,
 0.35355339059327373,
 0.35355339059327373,
 0.35355339059327373,
 0.35355339059327373]

#### 2. For each of the top 10 similar songs, get a list of the user purchases

In [33]:
## There are several ways to extract the user
## One way is to extract the row index
song_name = df.columns[1]
df[df[song_name] == 1]

,user,abba,ac/dc,adam green,aerosmith,afi,air,alanis morissette,alexisonfire,alicia keys,...,timbaland,tom waits,tool,tori amos,travis,trivium,u2,underoath,volbeat,yann tiersen
45,890,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99,1606,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
## One way is to extract the row index
list(df[df[song_name] == 1].index)

[45, 99]

In [35]:
## Another way is to extract the user id
list(df[df[song_name] == 1].iloc[:,0])

[890, 1606]

In [36]:
user_index = []

for song in range(1,285):
    song_name = df.columns[song]
    user_index.append(list(df[df[song_name] == 1].index))

In [37]:
user_index[:10]

[[45, 99],
 [21, 56, 64, 67, 89],
 [1],
 [66],
 [16, 52, 64, 82],
 [26, 32, 43, 44, 78],
 [32, 35],
 [5, 49, 57],
 [38, 59, 92, 96],
 [7, 13, 48]]

In [38]:
print('The user index shows that for sogn1, abba, user_index 45 and 99 purchased it.')

The user index shows that for sogn1, abba, user_index 45 and 99 purchased it.


#### 3. Calculate a recommendation score as follows: ∑(𝑝𝑢𝑟𝑐ℎ𝑎𝑠𝑒𝐻𝑖𝑠𝑡𝑜𝑟𝑦⋅𝑠𝑖𝑚𝑖𝑙𝑎𝑟𝑖𝑡𝑦𝑆𝑐𝑜𝑟𝑒)/∑𝑠𝑖𝑚𝑖𝑙𝑎𝑟𝑖𝑡𝑦𝑆𝑐𝑜𝑟𝑒

In [39]:
rec_score = df.copy()

In [40]:
## Calculating Recommendation Score:

## for each user each song:
for user in range(0,100):
    for song in range(1, 285):
        ## similar song for that song:
        similar_song = song_sim_list[song-1]
        similar_song_score = song_sim_cal[song-1]
        nominator = []
        denominator = sum(similar_song_score)
        
        ## we have 10 similar songs for each song:
        for s in range(0,10):
            song_index = df.columns.get_loc(similar_song[s])
            similar_user = user_index[song_index-1]
            purchase_history = 0
            
            ## for each song, we want to check if the user has listened to it:
            if user in similar_user:
                purchase_history = 1
            else:
                purchase_history = 0
            nominator.append(purchase_history*similar_song_score[s])
            
        score = sum(nominator)/denominator
        rec_score.iloc[user,song] = score

In [41]:
rec_score.head()

,user,abba,ac/dc,adam green,aerosmith,afi,air,alanis morissette,alexisonfire,alicia keys,...,timbaland,tom waits,tool,tori amos,travis,trivium,u2,underoath,volbeat,yann tiersen
0,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.147209,0.0,0.0,0.0,NaN,0.000000,0.0,0.088231,0.094411,0.0
1,33,0.000000,0.000000,1.000000,0.000000,0.208070,0.000000,0.0,0.094377,0.0,...,0.000000,0.0,0.0,0.0,NaN,0.000000,0.0,0.088231,0.000000,0.0
2,42,0.173849,0.206181,0.000000,0.072073,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,NaN,0.089999,0.0,0.000000,0.000000,0.0
3,51,0.000000,0.000000,0.188449,0.000000,0.081329,0.095548,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,NaN,0.000000,0.0,0.000000,0.000000,0.0
4,62,0.000000,0.000000,0.073010,0.000000,0.178129,0.000000,0.0,0.000000,0.0,...,0.217462,0.0,0.0,0.0,NaN,0.000000,0.0,0.101881,0.094411,0.0


#### 4. What are the top 5 song recommendations for user 1606?

In [42]:
user1606_recom = rec_score[rec_score['user']==1606].T
user1606_recom = user1606_recom.iloc[1:]
user1606_recom.columns = ['score']
user1606_recom

,score
abba,0.239023
ac/dc,0.000000
adam green,0.000000
aerosmith,0.000000
afi,0.000000
...,...
trivium,0.000000
u2,0.000000
underoath,0.000000
volbeat,0.000000


In [43]:
user1606_recom.sort_values(by = 'score', ascending = False).iloc[:5]

,score
elvis presley,0.289328
abba,0.239023
eric clapton,0.202740
frank sinatra,0.201139
howard shore,0.171749


In [44]:
listened = []

for i in range(0, len(user1606_song)):
    check = user1606_song[i] in user1606_recom.sort_values(by = 'score', ascending = False).iloc[:5].index
    if check == True:
        listened.append(user1606_song[i])

In [45]:
print('User 1606 has already listened ',listened)

User 1606 has already listened  ['abba', 'elvis presley', 'frank sinatra']


## 2. Conceptual Questions

### A. Name 2 other similarity measures that you can use instead of cosine similarity above.

#### Other than cosine similarity score, we can also use pearson correlation and Euclidean for similarity measures as those metrics can also present the spacial distance. Additionally, we can use NearestNeighbor method to find the similar vectors.

### B. What is needed to build a Content-Based Recommender system?

#### For content-based recommender system, we need to have an item/product's features to calculate the similarities. Content-based filtering uses item features to recommend other items similar to what the user likes, based on the users' previous feedbacks. Since we are comparing between an item and an item, we will need the item features. For example, if we are comparing two movies, we may want to know each movie's time length, language, category, and ratings. 

### C. Name 2 methods to evaluate your recommender system.

#### One method is the offline evaluation. In this case, we will look for low prediction errors by looking at the RMSE, MAE, high recall/catalog converge. Another way is the online evaluation. In this case, we want to search for customer lifetime values, apply some A/B testing to see if the recommendation system helps. 